In [2]:
import os
import scipy.io
import pandas as pd
import numpy as np


# Directory containing .mat files
folder_path = 'TFA/Data/'

# List all files in the directory
files = os.listdir(folder_path)

# Filter out .mat files
mat_files = [file for file in files if file.endswith('.mat')]


### Test and train split is done as follows: Combining all the subjects data and then splitting the data in 80:20 at random

In [3]:
dfs = []

for file in mat_files:
    file_path = os.path.join(folder_path, file)
    mat_data = scipy.io.loadmat(file_path)
    # Assuming your data is stored in 'tfaOut' key
    data = mat_data["tfaOut"]
    # print(np.shape(data))
    # Reshape data to 2D array
    reshaped_data = data.reshape(-1, data.shape[-1])
    df = pd.DataFrame(reshaped_data)
    # print(np.shape(df))
    label_column = np.zeros(len(df))  # Initialize with zeros
    label_column[:20500] = 0 # First 5000 rows get value 0
    label_column[-20500:] = 0  # Last 5000 rows get value 0
    label_column[20500:-20500] = 1
    df['label'] = label_column 
    dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
df = pd.concat(dfs, ignore_index=True)


In [4]:
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader


In [5]:
import tensorflow as tf

def split_features_target(data):
  # Assuming the label is in the last column
  features = data[:, :-1]  # All columns except the last
  target = data[:, -1]  # Last column
  features = torch.tensor(features)
  target = torch.tensor(target)
  return features, target

# Split features and target
X, y = split_features_target(df.to_numpy())
y = tf.expand_dims(y, axis=1)



### CNN

In [5]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define the input shape
input_shape = (10,)  # Assuming each sample has 10 features

# Define the CNN model
model = models.Sequential([
    layers.Reshape(input_shape + (1,), input_shape=input_shape),  # Reshape input to match (10, 1) for grayscale
    layers.Conv1D(32, kernel_size=3, activation='relu'),           # Convolutional layer with 32 filters
    layers.MaxPooling1D(pool_size=2),                              # Max pooling layer
    layers.Flatten(),                                              # Flatten layer
    layers.Dense(128, activation='relu'),                         # Dense layer with 128 neurons
    layers.Dropout(0.5),                                           # Dropout layer with dropout rate of 0.5
    layers.Dense(1, activation='sigmoid')                          # Output layer with 1 neuron and sigmoid activation
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Display the model summary
model.summary()
import numpy as np
from sklearn.model_selection import train_test_split

# Assuming X_train and y_train are your input and target data
X_np = np.array(X)
y_np = np.array(y)

# Split the data into training and validation sets
X_train_split, X_test_split, y_train_split, y_test_split = train_test_split(X_np, y_np, test_size=0.5, random_state=42)



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 10, 1)             0         
                                                                 
 conv1d (Conv1D)             (None, 8, 32)             128       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 4, 32)            0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 128)               16512     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                        

In [6]:
# Train the model
history = model.fit(X_train_split, y_train_split, epochs=1, batch_size=256)



from sklearn.metrics import precision_score, recall_score, confusion_matrix, accuracy_score

# Predict probabilities for each class
y_pred_prob = model.predict(X_test_split)

# Convert probabilities to class labels
y_pred = (y_pred_prob > 0.5).astype(int)

# Calculate precision
precision = precision_score(y_test_split, y_pred)

# Calculate recall
recall = recall_score(y_test_split, y_pred)

# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test_split, y_pred)

# Calculate accuracy
accuracy = accuracy_score(y_test_split, y_pred)

# Print the results
print("Precision:", precision)
print("Recall:", recall)
print("Confusion Matrix:\n", conf_matrix)
print("Accuracy:", accuracy)


2024-04-10 13:04:14.044200: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 363148480 exceeds 10% of free system memory.


35464/35464 [==============================] - 86s 2ms/step - loss: 0.1037 - accuracy: 0.9775


: 

: 

### NN

In [6]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define the input shape
input_shape = (10,)  # Assuming each sample has 10 features

# Define the neural network model
model = models.Sequential([
    layers.Dense(128, activation='relu', input_shape=input_shape),  # Dense layer with 64 neurons and ReLU activation
    layers.Dense(64, activation='relu'),                            # Dense layer with 32 neurons and ReLU activation
    layers.Dropout(0.1),                                             # Dropout layer with dropout rate of 0.5
    layers.Dense(1, activation='sigmoid')                            # Output layer with 1 neuron and sigmoid activation
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Display the model summary
model.summary()

import numpy as np
from sklearn.model_selection import train_test_split

# Assuming X_train and y_train are your input and target data
X_np = np.array(X)
y_np = np.array(y)

# Split the data into training and validation sets
X_train_split, X_test_split, y_train_split, y_test_split = train_test_split(X_np, y_np, test_size=0.5, random_state=42)

# Train the model
history = model.fit(X_train_split, y_train_split, epochs=1, batch_size=256, validation_data=(X_test_split, y_test_split))



Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 128)               1408      
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 1)                 65        
                                                                 
Total params: 9,729
Trainable params: 9,729
Non-trainable params: 0
_________________________________________________________________
35442/35464 [============================>.] - ETA: 0s - loss: 0.1021 - accuracy: 0.9775

: 

: 

In [ ]:
from sklearn.metrics import precision_score, recall_score, confusion_matrix, accuracy_score

# Predict probabilities for each class
y_pred_prob = model.predict(X_test_split)

# Convert probabilities to class labels
y_pred = (y_pred_prob > 0.5).astype(int)

# Calculate precision
precision = precision_score(y_test_split, y_pred)

# Calculate recall
recall = recall_score(y_test_split, y_pred)

# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test_split, y_pred)

# Calculate accuracy
accuracy = accuracy_score(y_test_split, y_pred)

# Print the results
print("Precision:", precision)
print("Recall:", recall)
print("Confusion Matrix:\n", conf_matrix)
print("Accuracy:", accuracy)
